아나콘다 에러

- AttributeError: module 'tweepy' has no attribute 'StreamListener' 에러 날 경우 !pip install tweepy==3.10.0

- Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl. 에러 날 경우 !pip install openpyxl

- Jpype 에러 https://ellun.tistory.com/46

## **1. 라이브러리 및 데이터 불러오기**

In [ ]:
#기본 라이브러리
import numpy as np
import pandas as pd
import os

#시각화 라이브러리
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#그래프 스타일 서식 지정
plt.style.use('default')
#그래프 한글 폰트
from matplotlib import font_manager, rc
plt.rc("font", family = "Malgun Gothic")
plt.rc("axes", unicode_minus = False)

#사이킷런
from sklearn.datasets import *
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score, precision_recall_curve, roc_curve, RocCurveDisplay
from sklearn.metrics import classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier, plot_importance

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler

# 텍스트 마이닝
from konlpy.tag import Okt # 형태소 분석에 사용할 konlpy 패키지의 Okt 클래스를 임포트하고 okt

from google.colab import drive
drive.mount('/content/gdrive/')
os.chdir('/content/gdrive/MyDrive/Colab Notebooks/data/')

train_df  = pd.read_excel('5movies.xlsx')
train_df .head(10)

## **2. 결측값 제거**

In [ ]:
# 댓글이 있는 항목만 담기(빈 댓글 삭제)
# text 컬럼이 non-null인 샘플만 train_df에 다시 저장
train_df = train_df[train_df['text'].notnull()]

# 수정된 train_df의 정보를 다시 확인
print(train_df.info())

# 분류 클래스의 구성을 확인
print(train_df['score'].value_counts())

## **3. 분석 불가능한 문자 제거**

In [ ]:
# 한글 외 문자 제거(옵션)
import re # 정규식을 사용하기 위해 re 모듈을 임포트
# ‘ㄱ ~‘힣’까지의 문자 제외한 나머지는 공백으로 치환, 영문: a-z | A-Z

train_df['text'] = train_df['text'].apply(lambda x : re.sub(r'[^ ㄱ-ㅣ가-힣]+', " ", x))
print(train_df.head())

# Train용 데이터셋의 정보를 재확인
print(train_df.info())
text = train_df['text'] # 시리즈 객체로 저장
score = train_df['score']

## **4. 데이터 셋 분리**

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(text, score , test_size=0.2, random_state=0)

## **5. 토큰화 및 TF-IDF 벡터화**

- 토큰화(tokenization)는 주어진 코퍼스(corpus)에서 토큰(token)이라 불리는 단위로 나눔
- 토큰화에서 고려해야 할 사항
    1. 구두점이나 특수문자를 단순 제외해도 무방한지 확인 (ex, Ph.D, AT&T, $2,000, 11/22/1990, 45.34)
    2. 줄임말과 단어 내 띄어쓰기 (we’re, thx, 고대, 영끌)
    3. 한국어는 어절 독립단어가 아니므로 형태소(자립, 의존) 분해가 필요
    4. 한국어는 띄어쓰기가 지켜지지 않고 있음
- TF-IDF 벡터화(단어 빈도-역 문서 빈도: Term Frequency-Inverse Document Frequency)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

okt = Okt()
tfv = TfidfVectorizer(tokenizer=okt.morphs, ngram_range=(1,2), min_df=3, max_df=0.9)
tfv.fit(x_train)
tfv_x_train = tfv.transform(x_train)
print(tfv_x_train)

## **6. 감성 분석 모델 구축**

In [ ]:
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import GridSearchCV # 하이퍼 파라미터 최적화

clf = LogisticRegression(random_state=0)
params = {'C': [15, 18, 19, 20, 22]}
grid_cv = GridSearchCV(clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv.fit(tfv_x_train, y_train)

# 최적의 평가 파라미터는 grid_cv.best_estimator_에 저장됨
print(grid_cv.best_params_, grid_cv.best_score_)# 가장 적합한 파라메터, 최고 정확도 확인

## **7. 분석 모델 평가**

In [ ]:
tfv_x_test = tfv.transform(x_test)

# test_predict = grid_cv.best_estimator_.score(tfv_test_x,test_y)

test_predict = grid_cv.best_estimator_.predict(tfv_x_test)

from sklearn.metrics import accuracy_score
print('감성 분류 모델의 정확도 : ',round(accuracy_score(y_test, test_predict), 3))

## **8. 감성 예측**

In [ ]:
input_text = '딱히 대단한 재미도 감동도 없는데 ~! 너무 과대 평과된 영화 중 하나'
#입력 텍스트에 대한 전처리 수행
input_text = re.compile(r'[ㄱ-ㅣ가-힣]+').findall(input_text)
print(input_text)
input_text = [" ".join(input_text)]
print(input_text)
# 입력 텍스트의 피처 벡터화
st_tfidf = tfv.transform(input_text)
print(st_tfidf)

# 최적 감성 분석 모델에 적용하여 감성 분석 평가
st_predict = grid_cv.best_estimator_.predict(st_tfidf)

#예측 결과 출력
if(st_predict == 0):
    print('예측 결과: ->> 부정 감성')
else :
    print('예측 결과: ->> 긍정 감성')

## **9. 출처**

https://velog.io/@changhtun1/Python-%EC%86%8C%EC%85%9C-%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%9D%98-%EA%B0%90%EC%84%B1-%EC%98%88%EC%B8%A1